In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim import Adam, SGD
import torchvision
import torchvision.transforms as transforms

import sys, os, math, csv
import argparse
import pandas as pd

In [2]:
lr=0.01
data='cifar10'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar10_vgg_ProActiv_OneTrainableSigma.pth'
resume = False

In [3]:
if data == 'cifar10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR10(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR10(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)
    
elif data == 'cifar100':
    nclass = 100
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.CIFAR100(root=root, train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testset = torchvision.datasets.CIFAR100(root=root, train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)
    
elif data == 'stl10':
    nclass = 10
    img_width = 32
    transform_train = transforms.Compose([
#         transforms.RandomCrop(32, padding=4),
#         transforms.RandomHorizontalFlip(),
        transforms.Resize((img_width,img_width)),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.Resize((img_width,img_width)),
        transforms.ToTensor(),
    ])
    trainset = torchvision.datasets.STL10(root=root, split='train', transform=transform_train, target_transform=None, download=True)
    testset = torchvision.datasets.STL10(root=root, split='test', transform=transform_test, target_transform=None, download=True)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=8)
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
if model == 'vgg':
    from models.vgg_dist import VGG_Dist
    net = nn.DataParallel(VGG_Dist(nclass, img_width=img_width).cuda())
    
net

DataParallel(
  (module): VGG_Dist(
    (conv1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv4): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    

In [5]:
if resume:
    print(f'==> Resuming from {model_out}')
    net.load_state_dict(torch.load(model_out))

In [6]:
cudnn.benchmark = True

In [7]:
criterion = nn.CrossEntropyLoss()

In [8]:
writefile_train = 'train.csv'
fieldnames = ['Epochs', 'Train Accuracy', 'ProbAct Sigma Value']
with open( writefile_train, 'w' ) as f:
    writer = csv.writer(f)
    writer.writerow(fieldnames)

In [9]:
def train(epoch):
    print('Epoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        pred = torch.max(outputs, dim=1)[1]
        correct += torch.sum(pred.eq(targets)).item()
        total += targets.numel()
    print(f'[TRAIN] Acc: {100.*correct/total:.3f}')
    with open( writefile_train, 'a' ) as f:
            writer = csv.writer(f)
            writer.writerow([epoch, 100.*correct/total, net.module.ProbActAF.weight.item()])

In [10]:
writefile_test = 'test.csv'
fieldnames = ['Epochs', 'Test Accuracy']
with open( writefile_test, 'w' ) as f:
    writer = csv.writer(f)
    writer.writerow(fieldnames)

In [11]:
def test(epoch):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print(f'[TEST] Acc: {100.*correct/total:.3f}')
        
        with open( writefile_test, 'a' ) as f:
            writer = csv.writer(f)
            writer.writerow([epoch, 100.*correct/total])

    # Save checkpoint after each epoch
    torch.save(net.state_dict(), model_out)

In [12]:
if data == 'cifar10':
    epochs = [50, 50, 50, 50]
elif data == 'cifar100':
    epochs = [50, 50, 50, 50]
elif data == 'stl10':
    epochs = [50, 50, 50, 50]

In [13]:
count = 0

In [14]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 18.638
[TEST] Acc: 23.930
Epoch: 1
[TRAIN] Acc: 32.754
[TEST] Acc: 42.290
Epoch: 2
[TRAIN] Acc: 52.962
[TEST] Acc: 55.740
Epoch: 3
[TRAIN] Acc: 66.196
[TEST] Acc: 56.710
Epoch: 4
[TRAIN] Acc: 73.270
[TEST] Acc: 53.690
Epoch: 5
[TRAIN] Acc: 78.042
[TEST] Acc: 64.710
Epoch: 6
[TRAIN] Acc: 81.462
[TEST] Acc: 73.330
Epoch: 7
[TRAIN] Acc: 84.388
[TEST] Acc: 76.930
Epoch: 8
[TRAIN] Acc: 86.232
[TEST] Acc: 77.840
Epoch: 9
[TRAIN] Acc: 88.278
[TEST] Acc: 73.320
Epoch: 10
[TRAIN] Acc: 89.584
[TEST] Acc: 77.310
Epoch: 11
[TRAIN] Acc: 91.104
[TEST] Acc: 80.730
Epoch: 12
[TRAIN] Acc: 92.318
[TEST] Acc: 81.420
Epoch: 13
[TRAIN] Acc: 93.716
[TEST] Acc: 82.690
Epoch: 14
[TRAIN] Acc: 94.664
[TEST] Acc: 80.180
Epoch: 15
[TRAIN] Acc: 94.996
[TEST] Acc: 80.190
Epoch: 16
[TRAIN] Acc: 95.740
[TEST] Acc: 82.730
Epoch: 17
[TRAIN] Acc: 96.486
[TEST] Acc: 79.720
Epoch: 18
[TRAIN] Acc: 96.334
[TEST] Acc: 80.940
Epoch: 19
[TRAIN] Acc: 96.882
[TEST] Acc: 82.110
Epoch: 20
[TRAIN] Acc: 97.036


[TRAIN] Acc: 100.000
[TEST] Acc: 87.180
Epoch: 166
[TRAIN] Acc: 100.000
[TEST] Acc: 87.210
Epoch: 167
[TRAIN] Acc: 100.000
[TEST] Acc: 87.120
Epoch: 168
[TRAIN] Acc: 100.000
[TEST] Acc: 87.050
Epoch: 169
[TRAIN] Acc: 100.000
[TEST] Acc: 87.170
Epoch: 170
[TRAIN] Acc: 100.000
[TEST] Acc: 87.130
Epoch: 171
[TRAIN] Acc: 100.000
[TEST] Acc: 87.210
Epoch: 172
[TRAIN] Acc: 100.000
[TEST] Acc: 87.110
Epoch: 173
[TRAIN] Acc: 100.000
[TEST] Acc: 87.210
Epoch: 174
[TRAIN] Acc: 100.000
[TEST] Acc: 87.180
Epoch: 175
[TRAIN] Acc: 99.998
[TEST] Acc: 87.060
Epoch: 176
[TRAIN] Acc: 100.000
[TEST] Acc: 87.220
Epoch: 177
[TRAIN] Acc: 99.998
[TEST] Acc: 87.160
Epoch: 178
[TRAIN] Acc: 100.000
[TEST] Acc: 87.150
Epoch: 179
[TRAIN] Acc: 99.998
[TEST] Acc: 87.050
Epoch: 180
[TRAIN] Acc: 100.000
[TEST] Acc: 87.120
Epoch: 181
[TRAIN] Acc: 100.000
[TEST] Acc: 87.120
Epoch: 182
[TRAIN] Acc: 100.000
[TEST] Acc: 87.090
Epoch: 183
[TRAIN] Acc: 100.000
[TEST] Acc: 86.940
Epoch: 184
[TRAIN] Acc: 99.996
[TEST] Acc: 87

In [15]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

merged=test.merge(train[['Epochs', 'Train Accuracy', 'ProbAct Sigma Value']], how='left', left_on='Epochs', right_on='Epochs')

merged.to_csv('VGG_CIFAR10_OneTrainableSigma.csv', index=False)

In [16]:
os.remove("train.csv")
os.remove("test.csv")

In [17]:
net.module.ProbActAF.weight.item()

-0.002663950202986598

In [2]:
lr=0.01
data='stl10'
root='./data/'
model='vgg'
model_out='./checkpoint/stl10_vgg_ProActiv_OneTrainableSigma.pth'
resume = False

In [14]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 11.780
[TEST] Acc: 13.863
Epoch: 1
[TRAIN] Acc: 11.840
[TEST] Acc: 10.000
Epoch: 2
[TRAIN] Acc: 12.860
[TEST] Acc: 10.525
Epoch: 3
[TRAIN] Acc: 13.400
[TEST] Acc: 10.637
Epoch: 4
[TRAIN] Acc: 16.160
[TEST] Acc: 10.675
Epoch: 5
[TRAIN] Acc: 18.420
[TEST] Acc: 13.613
Epoch: 6
[TRAIN] Acc: 20.560
[TEST] Acc: 17.025
Epoch: 7
[TRAIN] Acc: 24.100
[TEST] Acc: 19.962
Epoch: 8
[TRAIN] Acc: 24.040
[TEST] Acc: 22.650
Epoch: 9
[TRAIN] Acc: 25.280
[TEST] Acc: 23.675
Epoch: 10
[TRAIN] Acc: 25.800
[TEST] Acc: 22.113
Epoch: 11
[TRAIN] Acc: 28.820
[TEST] Acc: 25.738
Epoch: 12
[TRAIN] Acc: 30.600
[TEST] Acc: 31.075
Epoch: 13
[TRAIN] Acc: 33.800
[TEST] Acc: 33.212
Epoch: 14
[TRAIN] Acc: 36.240
[TEST] Acc: 31.500
Epoch: 15
[TRAIN] Acc: 37.800
[TEST] Acc: 36.913
Epoch: 16
[TRAIN] Acc: 39.040
[TEST] Acc: 33.375
Epoch: 17
[TRAIN] Acc: 39.840
[TEST] Acc: 38.513
Epoch: 18
[TRAIN] Acc: 43.760
[TEST] Acc: 35.475
Epoch: 19
[TRAIN] Acc: 46.040
[TEST] Acc: 35.688
Epoch: 20
[TRAIN] Acc: 48.040


[TRAIN] Acc: 100.000
[TEST] Acc: 61.888
Epoch: 166
[TRAIN] Acc: 100.000
[TEST] Acc: 61.987
Epoch: 167
[TRAIN] Acc: 100.000
[TEST] Acc: 61.950
Epoch: 168
[TRAIN] Acc: 100.000
[TEST] Acc: 61.925
Epoch: 169
[TRAIN] Acc: 100.000
[TEST] Acc: 61.900
Epoch: 170
[TRAIN] Acc: 100.000
[TEST] Acc: 61.962
Epoch: 171
[TRAIN] Acc: 100.000
[TEST] Acc: 61.825
Epoch: 172
[TRAIN] Acc: 100.000
[TEST] Acc: 61.962
Epoch: 173
[TRAIN] Acc: 100.000
[TEST] Acc: 61.913
Epoch: 174
[TRAIN] Acc: 100.000
[TEST] Acc: 61.812
Epoch: 175
[TRAIN] Acc: 100.000
[TEST] Acc: 62.000
Epoch: 176
[TRAIN] Acc: 100.000
[TEST] Acc: 61.962
Epoch: 177
[TRAIN] Acc: 100.000
[TEST] Acc: 62.013
Epoch: 178
[TRAIN] Acc: 100.000
[TEST] Acc: 61.950
Epoch: 179
[TRAIN] Acc: 100.000
[TEST] Acc: 62.087
Epoch: 180
[TRAIN] Acc: 100.000
[TEST] Acc: 62.125
Epoch: 181
[TRAIN] Acc: 100.000
[TEST] Acc: 62.075
Epoch: 182
[TRAIN] Acc: 100.000
[TEST] Acc: 62.087
Epoch: 183
[TRAIN] Acc: 100.000
[TEST] Acc: 62.087
Epoch: 184
[TRAIN] Acc: 100.000
[TEST] Acc

In [15]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

merged=test.merge(train[['Epochs', 'Train Accuracy', 'ProbAct Sigma Value']], how='left', left_on='Epochs', right_on='Epochs')

merged.to_csv('VGG_STL10_OneTrainableSigma.csv', index=False)

In [16]:
os.remove("train.csv")
os.remove("test.csv")

In [17]:
# print("Model's state_dict:")
# for param_tensor in net.state_dict():
#     print(param_tensor, "\t", net.state_dict()[param_tensor].size())

In [18]:
net.module.ProbActAF.weight.item()

2.415698872937355e-06

In [2]:
lr=0.01
data='cifar100'
root='./data/'
model='vgg'
model_out='./checkpoint/cifar100_vgg_ProActiv_OneTrainableSigma.pth'
resume = False

In [14]:
for epoch in epochs:
    optimizer = Adam(net.parameters(), lr=lr)
    for _ in range(epoch):
        train(count)
        test(count)
        count += 1
    lr /= 10

Epoch: 0
[TRAIN] Acc: 1.272
[TEST] Acc: 1.690
Epoch: 1
[TRAIN] Acc: 2.356
[TEST] Acc: 3.330
Epoch: 2
[TRAIN] Acc: 4.280
[TEST] Acc: 5.530
Epoch: 3
[TRAIN] Acc: 6.028
[TEST] Acc: 7.310
Epoch: 4
[TRAIN] Acc: 8.536
[TEST] Acc: 7.960
Epoch: 5
[TRAIN] Acc: 11.564
[TEST] Acc: 12.350
Epoch: 6
[TRAIN] Acc: 15.470
[TEST] Acc: 16.510
Epoch: 7
[TRAIN] Acc: 18.688
[TEST] Acc: 17.940
Epoch: 8
[TRAIN] Acc: 21.522
[TEST] Acc: 18.950
Epoch: 9
[TRAIN] Acc: 25.206
[TEST] Acc: 23.980
Epoch: 10
[TRAIN] Acc: 28.482
[TEST] Acc: 26.950
Epoch: 11
[TRAIN] Acc: 31.616
[TEST] Acc: 29.410
Epoch: 12
[TRAIN] Acc: 34.990
[TEST] Acc: 27.110
Epoch: 13
[TRAIN] Acc: 37.374
[TEST] Acc: 32.620
Epoch: 14
[TRAIN] Acc: 40.256
[TEST] Acc: 29.490
Epoch: 15
[TRAIN] Acc: 43.248
[TEST] Acc: 35.010
Epoch: 16
[TRAIN] Acc: 46.316
[TEST] Acc: 39.710
Epoch: 17
[TRAIN] Acc: 49.632
[TEST] Acc: 40.470
Epoch: 18
[TRAIN] Acc: 52.352
[TEST] Acc: 40.310
Epoch: 19
[TRAIN] Acc: 55.670
[TEST] Acc: 42.740
Epoch: 20
[TRAIN] Acc: 58.392
[TEST] Acc

[TEST] Acc: 50.750
Epoch: 167
[TRAIN] Acc: 99.966
[TEST] Acc: 50.670
Epoch: 168
[TRAIN] Acc: 99.964
[TEST] Acc: 50.640
Epoch: 169
[TRAIN] Acc: 99.932
[TEST] Acc: 50.880
Epoch: 170
[TRAIN] Acc: 99.930
[TEST] Acc: 50.850
Epoch: 171
[TRAIN] Acc: 99.958
[TEST] Acc: 50.720
Epoch: 172
[TRAIN] Acc: 99.942
[TEST] Acc: 50.780
Epoch: 173
[TRAIN] Acc: 99.956
[TEST] Acc: 50.720
Epoch: 174
[TRAIN] Acc: 99.950
[TEST] Acc: 50.740
Epoch: 175
[TRAIN] Acc: 99.944
[TEST] Acc: 50.720
Epoch: 176
[TRAIN] Acc: 99.948
[TEST] Acc: 50.760
Epoch: 177
[TRAIN] Acc: 99.942
[TEST] Acc: 50.740
Epoch: 178
[TRAIN] Acc: 99.940
[TEST] Acc: 50.810
Epoch: 179
[TRAIN] Acc: 99.956
[TEST] Acc: 50.770
Epoch: 180
[TRAIN] Acc: 99.940
[TEST] Acc: 50.620
Epoch: 181
[TRAIN] Acc: 99.964
[TEST] Acc: 50.670
Epoch: 182
[TRAIN] Acc: 99.946
[TEST] Acc: 50.870
Epoch: 183
[TRAIN] Acc: 99.958
[TEST] Acc: 50.680
Epoch: 184
[TRAIN] Acc: 99.954
[TEST] Acc: 50.710
Epoch: 185
[TRAIN] Acc: 99.946
[TEST] Acc: 50.710
Epoch: 186
[TRAIN] Acc: 99.940


In [15]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

merged=test.merge(train[['Epochs', 'Train Accuracy', 'ProbAct Sigma Value']], how='left', left_on='Epochs', right_on='Epochs')

merged.to_csv('VGG_CIFAR100_OneTrainableSigma.csv', index=False)

In [16]:
os.remove("train.csv")
os.remove("test.csv")

In [17]:
net.module.ProbActAF.weight.item()

-7.922627446532715e-06